<a href="https://colab.research.google.com/github/Tiabet/Project_Market/blob/master/ClassificationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
model_path = '/gdrive/MyDrive/bert_model.pth'

In [ ]:
import csv

# Replace 'data.tsv' with the path to your TSV file
tsv_file = '/gdrive/MyDrive/2199261531.tsv'

# Initialize an empty list to store the data from the TSV file
data = []

# Read the TSV file and store its content in the 'data' list
with open(tsv_file, 'r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        data.append(row)


In [ ]:
import pandas as pd

df = pd.DataFrame(data[1:], columns=data[0])

In [ ]:
df

,user_id,score,date,review,is_month,is_repurch
0,sati*****,5,23.08.01.,진짜 귀얍게 잘 갖고 놀아요,True,False
1,kb****,5,23.07.03.,강아지 최애 공입니다. 노랑색을 유난히 좋아하네요,True,False
2,kare*****,5,23.05.24.,노견 푸들이 공놀이를 너무 좋아하는데 이빨이 약해서 탄탄한 공은 잘 못 물더라구요....,False,False
3,cm****,5,23.05.18.,저희 강아지가 진짜 좋아하는 공이라 여러개 샀는데 좋아요,True,False
4,meji****,5,22.11.10.,온것중에 빨강이 골라서 줬는데 환장하구 놀아욤 !!\n막 뜯어버리는 애도 아니고 짱...,False,False
...,...,...,...,...,...,...
63,haha****,5,17.10.28.,잘삿어요 이건 뜯지도 못하고 애들이 잘가지고 노네요 ㅎㅎ\n잘산거같아요ㅎㅎ 견당 하...,False,False
64,haha****,5,17.10.28.,만족합니다,False,False
65,hyun*****,5,17.10.12.,너무너무 좋아하네요 5분만에 삑삑이 고장내고도 계속 가지고 놀아요 ^^,False,False
66,ooy_***,5,17.10.20.,배송도 빠르고 강아지가 좋아해요^^,False,False


In [ ]:
reviews = df['review']

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00


In [ ]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-6r3h5vxp/kobert-tokenizer_5a49d694b8a84791860378c5ea0575ca
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-6r3h5vxp/kobert-tokenizer_5a49d694b8a84791860378c5ea0575ca
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=213c1cf027a7c535d4343bc76e34870b76324c7dce91a2796484a752844ff949
  Stored in directory: /tmp/pip-ephem-wheel-cache-mjmix939/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
MAX_LEN = 128
BATCH_SIZE = 8

sentences = df['review']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks)
test_sampler = RandomSampler(test_data)
real_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
# Replace 'path_to_bert_model.pth' with the actual path to your saved BERT model
model_path = '/gdrive/MyDrive/bert_model.pth'


device = "cuda" if torch.cuda.is_available() else "cpu"
# Create an instance of the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', num_labels=2).to(device)

# Load the saved state_dict into the model
model.load_state_dict(torch.load(model_path, map_location=device),strict=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


_IncompatibleKeys(missing_keys=[], unexpected_keys=['bert.embeddings.position_ids'])

In [ ]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
import numpy as np

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

pred_result = []
label = []

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(real_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(real_dataloader)))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask = batch

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

        # 로스 구함
    logits = outputs.logits

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    # Get predictions
    predictions = np.argmax(logits, axis=1)

    # Collect predictions and labels
    pred_result.extend(predictions.tolist())

print("")


In [ ]:
df['pred_result'] = pred_result
df

,user_id,score,date,review,is_month,is_repurch,pred_result
0,sati*****,5,23.08.01.,진짜 귀얍게 잘 갖고 놀아요,True,False,1
1,kb****,5,23.07.03.,강아지 최애 공입니다. 노랑색을 유난히 좋아하네요,True,False,1
2,kare*****,5,23.05.24.,노견 푸들이 공놀이를 너무 좋아하는데 이빨이 약해서 탄탄한 공은 잘 못 물더라구요....,False,False,0
3,cm****,5,23.05.18.,저희 강아지가 진짜 좋아하는 공이라 여러개 샀는데 좋아요,True,False,1
4,meji****,5,22.11.10.,온것중에 빨강이 골라서 줬는데 환장하구 놀아욤 !!\n막 뜯어버리는 애도 아니고 짱...,False,False,1
...,...,...,...,...,...,...,...
63,haha****,5,17.10.28.,잘삿어요 이건 뜯지도 못하고 애들이 잘가지고 노네요 ㅎㅎ\n잘산거같아요ㅎㅎ 견당 하...,False,False,1
64,haha****,5,17.10.28.,만족합니다,False,False,1
65,hyun*****,5,17.10.12.,너무너무 좋아하네요 5분만에 삑삑이 고장내고도 계속 가지고 놀아요 ^^,False,False,1
66,ooy_***,5,17.10.20.,배송도 빠르고 강아지가 좋아해요^^,False,False,1


In [44]:
import datetime

# Get the current date and time
current_time = datetime.datetime.now()

result_path = '/gdrive/MyDrive/result_'+str(current_time)+'.csv'

df.to_csv(result_path, index=False, encoding='cp949', errors='ignore')